# QE-5.4.0 Input file ingester test

In [1]:
#### Copyright (C) 2018 Hud Wahab. All rights reserved.

## Imports

In [2]:
import os
import re
from math import sqrt
import numpy as np
import pandas as pd
from collections import Counter

## Class definitions

In [3]:
class PwInput():
    _pw = 'pw.x'
    
    def __init__(self, filename=None):
        """initial defs"""
        #kpoints
        self.ktype = "automatic"
        self.kpoints = [1,1,1]
        self.shiftk = [0,0,0]
        self.klist = []
        #dictionaries
        self.control = dict()
        self.system = dict()
        self.electrons = dict()
        self.ions = dict()
        self.cell = dict()
        self.atypes = dict()
        self.atoms = []
        self.cell_parameters = []
        self.cell_units = 'angstrom'
        self.atomic_pos_type = 'angstrom'
        
        #from a reference file
        if filename:
            f = open(filename,"r")
            self.file_name = filename 
            self.file_lines = f.readlines()
            self.singlespaces()
            self.remove_empty_lines()
            self.store(self.control,"control")     #read &control as dicts
            self.store(self.system,"system")      #read &system ..
            self.store(self.electrons,"electrons")   #read &electrons ..
            self.store(self.ions,"ions")        #read &ions .. 
            self.store(self.cell,"cell")        #read &cells ..
            #read ATOMIC_SPECIES
            self.read_atomicspecies()
            #read ATOMIC_POSITIONS
            self.read_atoms()
            #read K_POINTS
            self.read_kpoints()
            #read CELL_PARAMETERS
            self.read_cell_parameters()
        
    def get_atoms(self):
        """ Get the lattice parameters, postions of the atoms and chemical symbols
        """
        self.read_cell_parameters()
        cell = self.cell_parameters
        sym = [atom[0] for atom in self.atoms]
        pos = [atom[1] for atom in self.atoms]
        if self.atomic_pos_type == 'bohr':
            pos = car_red(pos,cell)
        return cell, pos, sym
    
    def get_masses(self):
        """ Get an array with the masses of all the atoms
        """
        masses = []
        for atom in self.atoms:
            atype = self.atypes[atom[0]]
            mass = float(atype[0])
            masses.append(mass)
        return masses
    
    def get_symmetry(self):
        """get the symmetry group of this system"""
        import spglib

        lat, positions, atypes = self.get_atoms()
        lat = np.array(lat)

        at = np.unique(atypes)
        an = dict(list(zip(at,list(range(len(at))))))
        atypes = [an[a] for a in atypes]

        cell = (lat,positions,atypes)

        spacegroup = spglib.get_spacegroup(cell,symprec=1e-5)
        return spacegroup
    
    def read_atomicspecies(self):
        """find ATOMIC_SPECIES keyword in file line by line"""
        lines = iter(self.file_lines)
        for line in lines:
            if "ATOMIC_SPECIES" in line:
                for i in range(int(self.system["ntyp"])):
                    atype, mass, psp = next(lines).split()
                    self.atypes[atype] = [mass,psp]  # e.g. atypes['C']=['12.017',C_1star-blabla.UPF]
                    
    def read_atoms(self):
        """find ATOMIC_POSITIONS keyword in file and read next lines"""
        lines = iter(self.file_lines)
        for line in lines:
            if "ATOMIC_POSITIONS" in line:
                atomic_pos_type = line
                self.atomic_pos_type = re.findall('([A-Za-z]+)',line)[-1]  #last word in line: angstrom or crystal
                for i in range(int(self.system["nat"])):
                    atype, x,y,z = next(lines).split()
                    self.atoms.append([atype,[float(i) for i in (x,y,z)]])
        self.atomic_pos_type = atomic_pos_type.replace('{','').replace('}','').strip().split()[1]
        
    def read_cell_parameters(self):
        ibrav = int(self.system['ibrav'])
        def rmchar(string,symbols): return ''.join([c for c in string if c not in symbols])

        if ibrav == 0:
            if 'celldm(1)' in list(self.system.keys()):
                a = float(self.system['celldm(1)'])
            else:
                a = 1
            lines = iter(self.file_lines)
            for line in lines:
                if "CELL_PARAMETERS" in line:
                    units = rmchar(line.strip(),'{}()').split()
                    self.cell_parameters = [[],[],[]]
                    if len(units) > 1:
                        self.cell_units = units[1]
                    else:
                        self.cell_units = 'bohr'
                    for i in range(3):
                        self.cell_parameters[i] = [ float(x)*a for x in next(lines).split() ]
            if self.cell_units == 'angstrom' or self.cell_units == 'bohr':
                if 'celldm(1)' in self.system: del self.system['celldm(1)']
            if 'celldm(1)' not in list(self.system.keys()):
                a = np.linalg.norm(self.cell_parameters[0])
        elif ibrav == 1:
            a = float(self.system['celldm(1)'])
            self.cell_parameters = [[  a,   0,   0],
                                    [  0,   a,   0],
                                    [  0,   0,   a]]
        elif ibrav == 2:
            a = float(self.system['celldm(1)'])
            self.cell_parameters = [[ -a/2,   0, a/2],
                                    [    0, a/2, a/2],
                                    [ -a/2, a/2,   0]]
        elif ibrav == 3:
            a = float(self.system['celldm(1)'])
            self.cell_parameters = [[ a/2,  a/2,  a/2],
                                    [-a/2,  a/2,  a/2],
                                    [-a/2, -a/2,  a/2]]
        elif ibrav == 4:
            a = float(self.system['celldm(1)'])
            c = float(self.system['celldm(3)'])
            self.cell_parameters = [[   a,          0,  0],
                                    [-a/2,sqrt(3)/2*a,  0],
                                    [   0,          0,c*a]]
        elif ibrav == 6:
            a = float(self.system['celldm(1)'])
            c = float(self.system['celldm(3)'])
            self.cell_parameters = [[  a,   0,   0],
                                    [  0,   a,   0],
                                    [  0,   0, c*a]]
        else:
            raise NotImplementedError('ibrav = %d not implemented'%ibrav)
        self.alat = a
        
    def read_kpoints(self):
        """find K_POINTS keyword in file and read next lines"""
        lines = iter(self.file_lines)
        for line in lines:
            if "K_POINTS" in line:
                if "automatic" in line:
                    self.ktype = "automatic"
                    vals = list(map(float, next(lines).split()))
                    self.kpoints, self.shiftk = vals[0:3], vals[3:6]
                #otherwise read a list
                else:
                    #read number of kpoints
                    nkpoints = int(next(lines).split()[0])
                    self.klist = []
                    self.ktype = ""
                    try:
                        lines_list = list(lines)
                        for n in range(nkpoints):
                            vals = lines_list[n].split()[:4]
                            self.klist.append( list(map(float,vals)) )
                    except IndexError:
                        print("wrong k-points list format")
                        exit()
    
    def remove_key(self,group,key):
        """ if a certain key exists in the group, remove it"""
        if key in list(group.items()):
            del group[key]
            
    def remove_empty_lines(self):
        """removes empty lines and \n in a file"""
        if not os.path.isfile(self.file_name):
            print("{} does not exist ".format(self.file_name))
            return
        with open(self.file_name) as filehandle:
            lines = filehandle.readlines()
        with open(self.file_name, 'w') as filehandle:
            lines = filter(lambda x: x.strip() and x.strip('\n'), lines)
            filehandle.writelines(lines)

    def run(self,filename,procs=1,folder='.'):
        """ this function is used to run this job locally
        """
        os.system('mkdir -p %s'%folder)
        self.write("%s/%s"%(folder,filename))
        if procs == 1:
            os.system('cd %s; OMP_NUM_THREADS=1 %s -inp %s > %s.log' % (folder,self._pw,filename,filename))
        else:
            os.system('cd %s; OMP_NUM_THREADS=1 mpirun -np %d %s -inp %s > %s.log' % (folder,procs,self._pw,filename,filename))

    def set_atoms_string(self,string):
        """
        set the atomic postions using string of the form
        Si 0.0 0.0 0.0
        Si 0.5 0.5 0.5
        """
        atoms_str = [line.strip().split() for line in string.strip().split('\n')]
        self.atoms = []
        for atype,x,y,z in atoms_str:
            self.atoms.append([atype,list(map(float,[x,y,z]))])
            
    def set_atoms(self,atoms):
        """ set the atomic postions using a Atoms datastructure from ase
        """
        # we will write down the cell parameters explicitly
        self.system['ibrav'] = 0
        if 'celldm(1)' in self.system: del self.system['celldm(1)']
        self.cell_parameters = atoms.get_cell()
        self.atoms = list(zip(atoms.get_chemical_symbols(),atoms.get_scaled_positions()))
        self.system['nat'] = len(self.atoms)
            
    def singlespaces(self):
        for line in self.file_lines:
            #' '.join(line.split())
            re.sub(' +',' ',line)
            
    def slicefile(self, keyword):
        """find lines after keyword between & and /"""
        lines = re.findall('&%s(?:.?)+\n((?:.+\n)+?)(?:\s+)?[\/&]'%keyword,"".join(self.file_lines),re.MULTILINE)
        return lines
    
    def store(self,group,name):
        """
        Save the variables specified in each of the groups on the structure, separated by =
        """
        for file_slice in self.slicefile(name):
            for keyword, value in re.findall('([a-zA-Z_0-9_\(\)]+)(?:\s+)?=(?:\s+)?([a-zA-Z\'"0-9_.-]+)',file_slice):
                group[keyword.strip()]=value.strip()
            
    def stringify_group(self, keyword, group):
        if group != {}:
            string='&%s\n' % keyword
            for keyword in sorted(group): # Py2/3 discrepancy in keyword order
                string += "%20s = %s\n" % (keyword, group[keyword])
            string += "/&end\n"
            return string
        else:
            return ''        
    
    def write(self,filename):
        f = open(filename,'w')
        f.write(str(self))
        f.close()

    def __str__(self):
        """Output the file in the form of a string"""
        string = ''
        string += self.stringify_group("control",self.control) #print control
        string += self.stringify_group("system",self.system) #print system
        string += self.stringify_group("electrons",self.electrons) #print electrons
        string += self.stringify_group("ions",self.ions) #print ions
        string += self.stringify_group("cell",self.cell) #print ions

        #print atomic species
        string += "ATOMIC_SPECIES\n"
        for atype in self.atypes:
            string += " %3s %8s %20s\n" % (atype, self.atypes[atype][0], self.atypes[atype][1])
        #print atomic positions
        string += "ATOMIC_POSITIONS { %s }\n"%self.atomic_pos_type
        for atom in self.atoms:
            string += "%3s %14.10lf %14.10lf %14.10lf\n" % (atom[0], atom[1][0], atom[1][1], atom[1][2])
        #print kpoints
        if self.ktype == "automatic":
            string += "K_POINTS { %s }\n" % self.ktype
            string += ("%3d"*6+"\n")%tuple(self.kpoints + self.shiftk)
        elif self.ktype == "crystal":
            string += "K_POINTS { %s }\n" % self.ktype
            string += "%d\n" % len(self.klist)
            for i in self.klist:
              string += ('%12.8lf '*4+'\n') % tuple(i)
        else:
            string += "K_POINTS { }\n"
            string += "%d\n" % len(self.klist)
            for i in self.klist:
                string += (("%12.8lf "*4)+"\n")%tuple(i)
        if self.system['ibrav'] == 0 or self.system['ibrav'] == '0':
            string += "CELL_PARAMETERS %s\n"%self.cell_units
            for i in range(3):
                string += ("%14.10lf "*3+"\n")%tuple(self.cell_parameters[i])
        return string
    
    def df_file(self):
        """combines all input parameters from one file to one dataframe """
        ctrl_df = pd.DataFrame.from_dict(self.control,orient='index')
        syst_df = pd.DataFrame.from_dict(self.system,orient='index')
        electron_df = pd.DataFrame.from_dict(self.electrons,orient='index')
        atom_species = pd.DataFrame.from_dict(self.atypes,orient='index',columns=['mass','psp'])
        atom_pos = pd.DataFrame(self.atoms,columns=['atom','xyz'])
        k_pts=pd.DataFrame(self.kpoints,index=['ka','kb','kc'],columns=['k'])
        cell_param = pd.DataFrame(self.cell_parameters,index=['a','b','c'],columns=['va','vb','vc'])
        groups = [ctrl_df,syst_df,electron_df,atom_species,atom_pos,k_pts,cell_param]
        return pd.concat(groups,keys=['ctrl_df','syst_df','electron_df','atom_species','atom_pos','k_pts','cell_param'])

def rename_duplicates(mylist):
    counts = Counter(mylist) # so we have: {'name':3, 'state':1, 'city':1, 'zip':2}
    for s,num in counts.items():
        if num > 1: # ignore strings that only appear once
            for suffix in range(1, num + 1): # suffix starts at 1 and increases by 1 each time
                mylist[mylist.index(s)] = s + '(' + str(suffix) + ')' # replace each appearance of s

## Tests

In [4]:
path = '../data/graphene/1.77/pwscf-Copy1.in'
data = PwInput('../data/graphene/1.77/pwscf-Copy1.in')
f1_n = os.path.basename(os.path.dirname(path))
# data.control['prefix'] = "'gc'"
# data.system['nat'] = 2
# data.system['ntyp'] = 2
# data.system['tot_charge'] = 1
# data.kpoints = [1,1,1]

print(data)

&control
         calculation = 'scf'
       etot_conv_thr = 1.d-6
       forc_conv_thr = 1.d-5
              outdir = '.
              prefix = 'graphene'
          pseudo_dir = '
        restart_mode = 'from_scratch'
/&end
&system
             degauss = 0.1
             ecutwfc = 80
               ibrav = 0
                 nat = 98
                ntyp = 2
         occupations = 'smearing'
            smearing = 'gaussian'
          tot_charge = 1
/&end
&electrons
            conv_thr = 1.d-10
      diago_thr_init = 1.d-4
     diagonalization = 'david'
         mixing_beta = 0.7
         mixing_mode = 'plain'
/&end
ATOMIC_SPECIES
   C  12.0107   C.pbe-mt_gipaw.UPF
 C_h  12.0107 C.star1s-pbe-mt_gipaw.UPF
ATOMIC_POSITIONS { crystal }
  C   0.0952444760   0.0476222380   0.5000000000
  C   0.0476222380   0.0952444760   0.5000000000
  C   0.2380910070   0.0476201850   0.5000000000
  C   0.1904781380   0.0952390740   0.5000000000
  C   0.3809535610   0.0476212180   0.5000000000
  C   0.33

In [5]:
pd.DataFrame(list(data.system.items()))

0           1
0        ibrav           0
1          nat          98
2         ntyp           2
3      ecutwfc          80
4  occupations  'smearing'
5     smearing  'gaussian'
6      degauss         0.1
7   tot_charge           1

In [6]:
syst_df = pd.DataFrame.from_dict(data.system,orient='index');syst_df

0
ibrav                 0
nat                  98
ntyp                  2
ecutwfc              80
occupations  'smearing'
smearing     'gaussian'
degauss             0.1
tot_charge            1

In [7]:
ctrl_df = pd.DataFrame.from_dict(data.control,orient='index');ctrl_df

0
calculation             'scf'
prefix             'graphene'
pseudo_dir                  '
outdir                     '.
restart_mode   'from_scratch'
etot_conv_thr           1.d-6
forc_conv_thr           1.d-5

In [8]:
electron_df = pd.DataFrame.from_dict(data.electrons,orient='index');electron_df

0
diagonalization  'david'
diago_thr_init     1.d-4
mixing_mode      'plain'
mixing_beta          0.7
conv_thr          1.d-10

In [9]:
atom_species = pd.DataFrame.from_dict(data.atypes,orient='index',columns=['mass','psp']);atom_species

mass                        psp
C    12.0107         C.pbe-mt_gipaw.UPF
C_h  12.0107  C.star1s-pbe-mt_gipaw.UPF

In [10]:
atom_pos = pd.DataFrame(data.atoms,columns=['atom','xyz']);atom_pos.head()

atom                              xyz
0    C  [0.095244476, 0.047622238, 0.5]
1    C  [0.047622238, 0.095244476, 0.5]
2    C  [0.238091007, 0.047620185, 0.5]
3    C  [0.190478138, 0.095239074, 0.5]
4    C  [0.380953561, 0.047621218, 0.5]

In [11]:
k_pts=pd.DataFrame(data.kpoints,index=['ka','kb','kc'],columns=['k']);k_pts

k
ka  1.0
kb  1.0
kc  1.0

In [12]:
cell_param = pd.DataFrame(data.cell_parameters,index=['a','b','c'],columns=['va','vb','vc']);cell_param

va         vb    vc
a  17.2970   0.000000   0.0
b  -8.6485  14.979641   0.0
c   0.0000   0.000000  10.0

In [13]:
groups = [ctrl_df,syst_df,electron_df,atom_species,atom_pos,k_pts,cell_param]

In [14]:
df1 = pd.concat(groups);df1

/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


0     mass                        psp atom  \
calculation               'scf'      NaN                        NaN  NaN   
prefix               'graphene'      NaN                        NaN  NaN   
pseudo_dir                    '      NaN                        NaN  NaN   
outdir                       '.      NaN                        NaN  NaN   
restart_mode     'from_scratch'      NaN                        NaN  NaN   
etot_conv_thr             1.d-6      NaN                        NaN  NaN   
forc_conv_thr             1.d-5      NaN                        NaN  NaN   
ibrav                         0      NaN                        NaN  NaN   
nat                          98      NaN                        NaN  NaN   
ntyp                          2      NaN                        NaN  NaN   
ecutwfc                      80      NaN                        NaN  NaN   
occupations          'smearing'      NaN                        NaN  NaN   
smearing             'gaussian'      NaN                        NaN  NaN   
degauss                     0.1      NaN                        NaN  NaN   
tot_charge                    1      NaN                        NaN  NaN   
diagonalization         'david'      NaN                        NaN  NaN   
diago_thr_init            1.d-4      NaN                        NaN  NaN   
mixing_mode             'plain'      NaN                        NaN  NaN   
mixing_beta                 0.7      NaN                        NaN  NaN   
conv_thr                 1.d-10      NaN                        NaN  NaN   
C                           NaN  12.0107         C.pbe-mt_gipaw.UPF  NaN   
C_h                         NaN  12.0107  C.star1s-pbe-mt_gipaw.UPF  NaN   
0                           NaN      NaN                        NaN    C   
1                           NaN      NaN                        NaN    C   
2                           NaN      NaN                        NaN    C   
3                           NaN      NaN                        NaN    C   
4                           NaN      NaN                        NaN    C   
5                           NaN      NaN                        NaN    C   
6                           NaN      NaN                        NaN    C   
7                           NaN      NaN                        NaN    C   
...                         ...      ...                        ...  ...   
74                          NaN      NaN                        NaN    C   
75                          NaN      NaN                        NaN    C   
76                          NaN      NaN                        NaN    C   
77                          NaN      NaN                        NaN    C   
78                          NaN      NaN                        NaN    C   
79                          NaN      NaN                        NaN    C   
80                          NaN      NaN                        NaN    C   
81                          NaN      NaN                        NaN    C   
82                          NaN      NaN                        NaN    C   
83                          NaN      NaN                        NaN    C   
84                          NaN      NaN                        NaN    C   
85                          NaN      NaN                        NaN    C   
86                          NaN      NaN                        NaN    C   
87                          NaN      NaN                        NaN    C   
88                          NaN      NaN                        NaN    C   
89                          NaN      NaN                        NaN    C   
90                          NaN      NaN                        NaN    C   
91                          NaN      NaN                        NaN    C   
92                          NaN      NaN                        NaN    C   
93                          NaN      NaN                        NaN    C   
94                          NaN      NaN                        NaN    C   
95

In [15]:
path2 = '../data/rnklelite/05.w2.34.5.25/pwscf.in'
data2 = PwInput(path2)
f2_n = os.path.basename(os.path.dirname(path2))
datas = [data,data2]
datas

[<__main__.PwInput at 0x10efd0940>, <__main__.PwInput at 0x10f02f3c8>]

In [16]:
f1 = data.df_file();f1

/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


0     mass  \
ctrl_df      calculation               'scf'      NaN   
             prefix               'graphene'      NaN   
             pseudo_dir                    '      NaN   
             outdir                       '.      NaN   
             restart_mode     'from_scratch'      NaN   
             etot_conv_thr             1.d-6      NaN   
             forc_conv_thr             1.d-5      NaN   
syst_df      ibrav                         0      NaN   
             nat                          98      NaN   
             ntyp                          2      NaN   
             ecutwfc                      80      NaN   
             occupations          'smearing'      NaN   
             smearing             'gaussian'      NaN   
             degauss                     0.1      NaN   
             tot_charge                    1      NaN   
electron_df  diagonalization         'david'      NaN   
             diago_thr_init            1.d-4      NaN   
             mixing_mode             'plain'      NaN   
             mixing_beta                 0.7      NaN   
             conv_thr                 1.d-10      NaN   
atom_species C                           NaN  12.0107   
             C_h                         NaN  12.0107   
atom_pos     0                           NaN      NaN   
             1                           NaN      NaN   
             2                           NaN      NaN   
             3                           NaN      NaN   
             4                           NaN      NaN   
             5                           NaN      NaN   
             6                           NaN      NaN   
             7                           NaN      NaN   
...                                      ...      ...   
             74                          NaN      NaN   
             75                          NaN      NaN   
             76                          NaN      NaN   
             77                          NaN      NaN   
             78                          NaN      NaN   
             79                          NaN      NaN   
             80                          NaN      NaN   
             81                          NaN      NaN   
             82                          NaN      NaN   
             83                          NaN      NaN   
             84                          NaN      NaN   
             85                          NaN      NaN   
             86                          NaN      NaN   
             87                          NaN      NaN   
             88                          NaN      NaN   
             89                          NaN      NaN   
             90                          NaN      NaN   
             91                          NaN      NaN   
             92                          NaN      NaN   
             93                          NaN      NaN   
             94                          NaN      NaN   
             95                          NaN      NaN   
             96                          NaN      NaN   
             97                          NaN      NaN   
k_pts        ka                          NaN      NaN   
             kb                          NaN      NaN   
             kc                          NaN      NaN   
cell_param   a                           NaN      NaN   
             b                           NaN      NaN   
             c                           NaN      NaN   

                                                    psp atom  \
ctrl_df      calculation                            NaN  NaN   
             prefix                                 NaN  NaN   
             pseudo_dir                             NaN  NaN   
             outdir                                 NaN  NaN   
             restart_mode                           NaN  NaN   
             etot_conv_thr                          NaN  NaN   
             forc_conv_thr                          NaN  NaN

In [17]:
f2 = data2.df_file()


/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


In [18]:
x = pd.concat([f1,f2],keys =[f1_n,f2_n],axis=1)

In [19]:
paths= '../data/'
dirs = []
filenames = []
for dirpath,_,fnames in os.walk(paths):
    dirs.append(dirpath)
    filenames.append(fnames)

In [20]:
folders = []
for j in dirs:
    if os.path.exists(f'{j}/pwscf.in'):
        folders.append(os.path.basename(j))
rename_duplicates(folders)

In [21]:
len(set(folders))==len(folders)

True

In [22]:
dfs = []
for j in dirs:
    if os.path.exists(f'{j}/pwscf.in'):
        class_var = PwInput(f'{j}/pwscf.in')
        dfs.append(class_var.df_file())
data = pd.concat(dfs,keys = folders,axis=1)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


In [23]:
data.head()

1.h.34.5.25                 \
                     0 mass  psp atom   
atom_pos 0         NaN  NaN  NaN   Cu   
         1         NaN  NaN  NaN   Cu   
         2         NaN  NaN  NaN   Cu   
         3         NaN  NaN  NaN   Cu   
         4         NaN  NaN  NaN   Cu   

                                                                     \
                                                xyz   k  va  vb  vc   
atom_pos 0  [0.000783646, -0.001992654, 7.36702207] NaN NaN NaN NaN   
         1   [1.845099279, 1.704434127, 7.36726565] NaN NaN NaN NaN   
         2  [3.689797387, -0.001497907, 7.36697526] NaN NaN NaN NaN   
         3  [5.534498526, 1.704579609, 7.367287497] NaN NaN NaN NaN   
         4  [0.000723006, 3.408571399, 7.366869818] NaN NaN NaN NaN   

           3.O.34.5.25 ... xcooh.rib.34.5.25 7.rib.34.5.25                 \
                     0 ...                vc             0 mass  psp atom   
atom_pos 0         NaN ...               NaN           NaN  NaN  NaN   Cu   
         1         NaN ...               NaN           NaN  NaN  NaN   Cu   
         2         NaN ...               NaN           NaN  NaN  NaN   Cu   
         3         NaN ...               NaN           NaN  NaN  NaN   Cu   
         4         NaN ...               NaN           NaN  NaN  NaN   Cu   

                                                                      
                                                 xyz   k  va  vb  vc  
atom_pos 0          [0.0, -0.000829132, 7.386830164] NaN NaN NaN NaN  
         1   [1.844443097, 1.707942874, 7.399365766] NaN NaN NaN NaN  
         2  [3.689036991, -0.001134632, 7.386342761] NaN NaN NaN NaN  
         3   [5.533630886, 1.707942874, 7.399365766] NaN NaN NaN NaN  
         4            [0.0, 3.41055262, 7.417564754] NaN NaN NaN NaN  

[5 rows x 1377 columns]

In [24]:
folders

['1.h.34.5.25',
 '3.O.34.5.25',
 '2.cooh.34.5.25',
 '7.h.34.5.25',
 '3.r.34.5.25',
 '5.O.34.5.25',
 '2.hh.34.5.25',
 '2.O.34.5.25',
 'O.34.5.25',
 '1.no.cooh.34.5.25',
 '1x.34.5.25',
 'coo.34.5.25',
 '2.r.34.5.25',
 '7.hh.34.5.25',
 '4.O.34.5.25',
 'cooh.34.5.25',
 '10.OH.34.5.25',
 'OH.34.5.25',
 '7.r.34.5.25',
 '1.O.34.5.25',
 '1.hh.34.5.25',
 '15.OH.34.5.25',
 '7.O.34.5.25',
 '1.r.34.5.25',
 '0.cooh.34.5.25',
 '1.OH.34.5.25',
 '7.cooh.34.5.25',
 'h.34.5.25',
 '2.h.34.5.25',
 'hh.34.5.25',
 '1.cooh.34.5.25',
 '6.O.34.5.25',
 '0.iC2H4.34.5.25',
 '0.iCO.34.5.25',
 '9.iCO2.34.5.25',
 '10.iCH2.34.5.25',
 '0.iCH2.34.5.25(1)',
 '9.iCO.34.5.25',
 'i.OH.34.5.25',
 '15.iC2H4.34.5.25',
 'i.H2.34.5.25',
 'iCH4.34.5.25',
 '15.iCH4.34.5.25',
 '9.iC2H4.34.5.25',
 'i.O2.34.5.25',
 'iCO.34.5.25',
 '9.iCH2.34.5.25',
 '10.iCO2.34.5.25',
 '1.i.OH.34.5.25',
 '0.iC.34.5.25',
 'iO.34.5.25',
 '0.iCH4.34.5.25',
 '10.iCO.34.5.25',
 '15.iCO2.34.5.25',
 'iC.34.5.25',
 'iCH2.34.5.25',
 '10.iC.34.5.25',
 '1.iCO2